<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup</a></span></li><li><span><a href="#Grouping-Data-Frames" data-toc-modified-id="Grouping-Data-Frames-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Grouping Data Frames</a></span><ul class="toc-item"><li><span><a href="#Collapsing-data" data-toc-modified-id="Collapsing-data-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Collapsing data</a></span></li><li><span><a href="#Transorming-data-by-'group'" data-toc-modified-id="Transorming-data-by-'group'-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Transorming data by 'group'</a></span></li></ul></li><li><span><a href="#Restructuring-Data" data-toc-modified-id="Restructuring-Data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Restructuring Data</a></span><ul class="toc-item"><li><span><a href="#Pivoting" data-toc-modified-id="Pivoting-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Pivoting</a></span></li><li><span><a href="#Melt" data-toc-modified-id="Melt-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Melt</a></span></li></ul></li><li><span><a href="#Combining-Data-Frames" data-toc-modified-id="Combining-Data-Frames-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Combining Data Frames</a></span><ul class="toc-item"><li><span><a href="#Merge" data-toc-modified-id="Merge-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Merge</a></span></li><li><span><a href="#Concatenate" data-toc-modified-id="Concatenate-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Concatenate</a></span></li></ul></li></ul></div>

# Setup

In [1]:
import pandas as pd
import numpy as np

In [2]:
D = {
    "x": ["bar", "bar", "bar", "foo", "foo", "foo"],
    "y": ["a", "b", "c", "a", "b", "c"],
    "z": [2, 3, 5, 7, 1, 2]
}
DF = pd.DataFrame(D)

In [3]:
DF

,x,y,z
0,bar,a,2
1,bar,b,3
2,bar,c,5
3,foo,a,7
4,foo,b,1
5,foo,c,2


# Grouping Data Frames

## Collapsing data

In [4]:
DF1 = pd.DataFrame(DF.groupby('x')['z'].aggregate(np.sum))

In [5]:
DF1

,z
x,
bar,10
foo,10


## Transorming data by 'group'

In [6]:
DF2 = pd.DataFrame(DF.groupby('x')['z'].transform(np.cumsum))

In [7]:
DF2

,z
0,2
1,5
2,10
3,7
4,8
5,10


In [8]:
DF.loc[:, 'z_cumsum'] = DF.groupby('x')['z'].transform(np.cumsum)

In [9]:
DF

,x,y,z,z_cumsum
0,bar,a,2,2
1,bar,b,3,5
2,bar,c,5,10
3,foo,a,7,7
4,foo,b,1,8
5,foo,c,2,10


In [10]:
DF.loc[:, 'z_ave'] = DF.groupby('x')['z'].transform(np.mean)
DF.loc[:, 'z_std'] = DF.groupby('x')['z'].transform(np.std)
DF

,x,y,z,z_cumsum,z_ave,z_std
0,bar,a,2,2,3.333333,1.527525
1,bar,b,3,5,3.333333,1.527525
2,bar,c,5,10,3.333333,1.527525
3,foo,a,7,7,3.333333,3.214550
4,foo,b,1,8,3.333333,3.214550
5,foo,c,2,10,3.333333,3.214550


In [11]:
DF.loc[:, 'z_score'] = (DF['z'] - DF['z_ave']) / DF['z_std']

In [12]:
DF

,x,y,z,z_cumsum,z_ave,z_std,z_score
0,bar,a,2,2,3.333333,1.527525,-0.872872
1,bar,b,3,5,3.333333,1.527525,-0.218218
2,bar,c,5,10,3.333333,1.527525,1.091089
3,foo,a,7,7,3.333333,3.214550,1.140647
4,foo,b,1,8,3.333333,3.214550,-0.725866
5,foo,c,2,10,3.333333,3.214550,-0.414781


# Restructuring Data 

In [13]:
DF = DF.loc[:, ['x', 'y', 'z']]

In [14]:
DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 3 columns):
x    6 non-null object
y    6 non-null object
z    6 non-null int64
dtypes: int64(1), object(2)
memory usage: 224.0+ bytes


In [15]:
DF

,x,y,z
0,bar,a,2
1,bar,b,3
2,bar,c,5
3,foo,a,7
4,foo,b,1
5,foo,c,2


## Pivoting

In [16]:
DF.pivot(index='x', columns='y', values='z')

y,a,b,c
x,,,
bar,2,3,5
foo,7,1,2


In [17]:
DF3 = DF.pivot(index='x', columns='y', values='z')

## Melt

In [19]:
DF4 = DF3.reset_index().melt(
    id_vars=DF3.index.name, value_vars=["a", "b", "c"], value_name="z")

In [20]:
DF4

,x,y,z
0,bar,a,2
1,foo,a,7
2,bar,b,3
3,foo,b,1
4,bar,c,5
5,foo,c,2


# Combining Data Frames

## Merge

In [21]:
D0 = {"x": ["a", "b", "c"], "y": [3, 4, 2]} 
D1 = {"x": ["a", "b", "c"], "z": [1, 9, np.nan]}
DF0 = pd.DataFrame(D0)
DF1 = pd.DataFrame(D1)

In [23]:
DF0

,x,y
0,a,3
1,b,4
2,c,2


In [24]:
DF1

,x,z
0,a,1.0
1,b,9.0
2,c,NaN


In [25]:
DF2 = pd.merge(DF0, DF1, how="inner", on="x")

In [26]:
DF2

,x,y,z
0,a,3,1.0
1,b,4,9.0
2,c,2,NaN


In [27]:
DF0.set_index("x", inplace=True)
DF1.set_index("x", inplace=True)

In [28]:
DF0

,y
x,
a,3
b,4
c,2


In [29]:
DF1

,z
x,
a,1.0
b,9.0
c,NaN


In [30]:
DF2 = pd.merge(DF0, DF1, left_index=True, right_index=True)

In [31]:
DF2

,y,z
x,,
a,3,1.0
b,4,9.0
c,2,NaN


In [33]:
DF1.drop("c", axis=0, inplace=True)

In [34]:
DF1

,z
x,
a,1.0
b,9.0


In [35]:
DF0

,y
x,
a,3
b,4
c,2


In [36]:
DF0.reset_index(inplace=True)
DF1.reset_index(inplace=True)

In [37]:
pd.merge(DF0, DF1, on="x", how="left")

,x,y,z
0,a,3,1.0
1,b,4,9.0
2,c,2,NaN


In [38]:
pd.merge(DF0, DF1, on="x", how="right")

,x,y,z
0,a,3,1.0
1,b,4,9.0


## Concatenate

In [39]:
D0 = {"x": ["a", "b", "c"], "y": [3, 4, 2]} 
D1 = {"x": ["d", "e", "f"], "y": [1, 9, np.nan]}
DF0 = pd.DataFrame(D0)
DF1 = pd.DataFrame(D1)

In [40]:
DF0

,x,y
0,a,3
1,b,4
2,c,2


In [41]:
DF1

,x,y
0,d,1.0
1,e,9.0
2,f,NaN


In [44]:
DF2 = pd.concat([DF0, DF1], ignore_index=True)

In [45]:
DF2

,x,y
0,a,3.0
1,b,4.0
2,c,2.0
3,d,1.0
4,e,9.0
5,f,NaN
